In [1]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

import os
import pandas as pd
import numpy as np
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
from matplotlib import dates, ticker

from ladybug.epw import EPW
import oshe as tc

Failed to load paths from C:\Users\tgerrish\Miniconda3\lib\site-packages\honeybee\config.json.
Path to radiance is set to: c:/radiance


In [2]:
# Inputs
idd_file = r"C:\openstudio-2.9.0\EnergyPlus\Energy+.idd"
epw_file = r"C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\UWG_ARE_AZ_Abu.Dhabi-Bateen.Exec.AP.412160_TMYx_URBAN_02.epw"
idx = pd.date_range(start="2018-01-01 00:30:00", freq="60T", periods=8760, closed="left")
# Load weatherfile data

# Load epw
epw = EPW(epw_file)

# Get climate variables
dbt = np.roll(np.array(epw.dry_bulb_temperature.values), -1)
rh = np.roll(np.array(epw.relative_humidity.values), -1)
ws = np.roll(np.array(epw.wind_speed.values), -1)
hir = np.roll(np.array(epw.horizontal_infrared_radiation_intensity.values), -1)

# Calculate hourly solar altitudes
sun_altitude = np.roll(tc.helpers.sun_altitude(epw_file), -1)

In [27]:
# NV method processing

_dir = r"C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\CapitalGardens"

# Load Openfield MRT and calculate UTCI using same method as NV MRT > UTCI is generated
of_mrt = pd.read_csv(r"C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\openfield_nv.mrt", header=None).values.T
of_utci = tc.utci.universal_thermal_climate_index(dbt, of_mrt, ws, rh)[0]

# Load MRT
mean_radiant_temperature = pd.read_csv(_dir+"\MRT.csv", header=None).values.T

# Load PTS
points = pd.read_csv(_dir+"\points.csv", header=None).values

# Load focus pts
focus_pts = pd.read_csv(_dir + "\\focus_pts.csv", header=None).values.T[0]

# Set radiance geometries
rad_files = []
for root, dirs, files in os.walk(_dir):
    for file in files:
        if file.endswith("opq.rad") or file.endswith("glz.rad"):
             rad_files.append(os.path.join(root, file))

# Calculate UTCI
universal_thermal_climate_index = tc.utci.utci_parallel(
        threads=8, 
        air_temperature=dbt, 
        mean_radiant_temperature=mean_radiant_temperature.T, 
        wind_speed=ws, 
        relative_humidity=rh
    )

# Create UTCI object
utci = tc.plot.UTCI(of_utci, universal_thermal_climate_index, points)

# Plot the results
utci.generate_plots(rad_files=rad_files, focus_pts=focus_pts, boundary=None, plot_directory=_dir+"\plots2")

Universal thermal climate index calculated [0.10 seconds]
Plot saved to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\ChamberOfCommerce\plots2\Area2\openfield_comfortheatmap.png
Plot saved to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\ChamberOfCommerce\plots2\Area2\comfortable_hours_may.png
Plot saved to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\ChamberOfCommerce\plots2\Area2\comfortable_hours_october.png
Plot saved to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\ChamberOfCommerce\plots2\Area2\comfortable_hours_annual.png
Plot saved to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\ChamberOfCommerce\plots2\Area2\context.png
Plot saved to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\ChamberOfCommerce\plots2\Area2\context_ptlabel.png
Plot saved to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\ChamberOfCommerce\plots2\Area

In [28]:
# SC method processing

# Open-field MRT and UTCI calculation

# Define ground material
gnd_mat = tc.material.material_dict["CONCRETE"]
ground_zone = tc.geometry.Ground(gnd_mat, xy=5, depth=1.5, subsurface_size=5)

# Calculate ground surface temperature
of_srf_temp = tc.energyplus.run_energyplus(epw_file, idd_file, ground=ground_zone, shades=None, run=True)

# Calculate incident solar direct and diffuse radiation
of_dir_rad, of_dif_rad = tc.radiance.run_radiance(epw_file, ground=ground_zone, shades=None, run=True)

# Calculate MRT
of_mrt = tc.mrt.mean_radiant_temperature(surrounding_surfaces_temperature=of_srf_temp[0], horizontal_infrared_radiation_intensity=hir, diffuse_horizontal_solar=of_dif_rad.T[0], direct_normal_solar=of_dir_rad.T[0], sun_altitude=sun_altitude, ground_reflectivity=gnd_mat.reflectivity, sky_exposure=1)[0]

# Calculate UTCI
of_utci = tc.utci.universal_thermal_climate_index(dbt, of_mrt, ws, rh)

Ground surface temperature simulation completed
EnergyPlus results loaded
Writing recipe contents to: C:\Users\tgerrish\AppData\Local\Temp\openfield\gridbased_annual
Calculating solar values...
# Number of sun up hours: 4423
Writing sun matrix to C:\Users\tgerrish\AppData\Local\Temp\openfield\gridbased_annual\sky\sunmtx.smx
Direct and diffuse solar radiation simulation completed
Radiance results loaded
Mean radiant temperature calculated [0.11 seconds]
Universal thermal climate index calculated [0.14 seconds]


In [29]:
_dir = r"C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\Municipality\MUNICIPALITY"

# Load case recipe
recipe = _dir + "\\recipe.json"
reload = False

# Load radiance geometries
# Set radiance geometries
rad_files = []
for root, dirs, files in os.walk(_dir):
    for file in files:
        if file.endswith("opq.rad") or file.endswith("glz.rad"):
             rad_files.append(os.path.join(root, file))

# Load focus pts
focus_pts = pd.read_csv(_dir + "\\focus_pts.csv", header=None).values.T[0]

# Create output directory for generated images
output_directory = os.path.dirname(os.path.dirname(recipe))
plot_directory = os.path.join(os.path.dirname(recipe), "plots2")
if not os.path.exists(plot_directory):
    os.mkdir(plot_directory)

# Load JSON
d = tc.helpers.load_json(recipe)

# Load surface view factors
surface_view_factors = np.array(d["surface_view_factors"])

# Load sky view factors
sky_view_factors = np.array([[i] for i in d["sky_view_factors"]]) / 100 * 0.5

# Load ground reflectivities
ground_reflectivities = np.array([[i] for i in d["ground_reflectivities"]])

# Load plot tricontour plot 
boundary = np.array(d["boundary"])[:, :-1]

# Load points
points = np.array(d["points"])

if not reload:
    # Read EnergyPlus annual hourly surface temperature results
    surface_temperatures = tc.helpers.load_energyplus_results(d["eplus_results"])

    # Read Radiance simulation annual hourly incident radiation results
    radiation_direct, radiation_diffuse = tc.helpers.load_radiance_results(d["radiation_results"])
    
    # Calculate generic sky temperature
    skytemp = tc.oshe.sky_temperature(hir)

    # Append sky temperature and air temperature onto end of surface temperatures
    all_srf_temps = np.vstack([surface_temperatures, skytemp, dbt])

    # Append sky view factor to end of surface view factors
    all_vfs = np.hstack([surface_view_factors, sky_view_factors])

    # Add view factor to end of each group, bringing total to 1
    makeups = np.where(1 - all_vfs.sum(axis=1) <= 0, 0, 1 - all_vfs.sum(axis=1))
    all_vfs = np.vstack([all_vfs.T, makeups]).T

    # Normalise view factors to 0-1
    all_vfs = all_vfs.T * (1 / all_vfs.sum(axis=1))

    # Calculate overall surrounding surface temperature using view factors and individual surface temperatures
    surrounding_surface_temperatures = np.power(np.matmul(all_vfs.T, np.power(all_srf_temps.T + 273.15, 4).T), 0.25) - 273.15
    print("View-factored surface temperatures calculated")

EnergyPlus results loaded
Radiance results loaded
View-factored surface temperatures calculated


In [30]:
# Calculate MRT and write to file

mrt_path = os.path.join(os.path.dirname(recipe), "result.mrt")
if not reload:
    mean_radiant_temperature = tc.mrt.mrt_parallel(
        threads=8, 
        surrounding_surfaces_temperature=surrounding_surface_temperatures, 
        horizontal_infrared_radiation_intensity=hir, 
        diffuse_horizontal_solar=radiation_diffuse, 
        direct_normal_solar=radiation_direct, 
        sun_altitude=sun_altitude, 
        ground_reflectivity=ground_reflectivities, 
        sky_exposure=sky_view_factors, 
        radiance=True
    )

    # Write to file
    df_mrt = pd.DataFrame(mean_radiant_temperature.T)
    df_mrt.round(6).to_csv(mrt_path, index=False)
    print("MRT calculated and written to {}".format(mrt_path))
else:
    mean_radiant_temperature = pd.read_csv(mrt_path).values.T
    print("MRT reloaded from {}".format(mrt_path))

MRT calculated and written to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\StreetFacelifts\STREETFACELIFTS_P1\result.mrt


In [31]:
# Calculate UTCI and write to file

utci_path = os.path.join(os.path.dirname(recipe), "result.utci")
if not reload:
    universal_thermal_climate_index = tc.utci.utci_parallel(
        threads=8, 
        air_temperature=dbt, 
        mean_radiant_temperature=mean_radiant_temperature.T, 
        wind_speed=ws, 
        relative_humidity=rh
    )

    # Write to file
    df_utci = pd.DataFrame(universal_thermal_climate_index.T)
    df_utci.round(6).to_csv(utci_path, index=False)
    print("UTCI calculated and written to {}".format(utci_path))
else:
    universal_thermal_climate_index = pd.read_csv(utci_path).values.T
    print("UTCI reloaded from {}".format(utci_path))

UTCI calculated and written to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\StreetFacelifts\STREETFACELIFTS_P1\result.utci


In [32]:
# Construct results object
utci = tc.plot.UTCI(of_utci, universal_thermal_climate_index, points)

# Plot the results
utci.generate_plots(rad_files=rad_files, focus_pts=focus_pts, boundary=boundary, plot_directory=_dir+"\plots2")

Plot saved to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\StreetFacelifts\STREETFACELIFTS_P1\plots2\openfield_comfortheatmap.png
Plot saved to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\StreetFacelifts\STREETFACELIFTS_P1\plots2\comfortable_hours_may.png
Plot saved to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\StreetFacelifts\STREETFACELIFTS_P1\plots2\comfortable_hours_october.png
Plot saved to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\StreetFacelifts\STREETFACELIFTS_P1\plots2\comfortable_hours_annual.png
Plot saved to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\StreetFacelifts\STREETFACELIFTS_P1\plots2\context.png
Plot saved to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\StreetFacelifts\STREETFACELIFTS_P1\plots2\context_ptlabel.png
Plot saved to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\StreetFacelifts\STREE

Plot saved to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\StreetFacelifts\STREETFACELIFTS_P1\plots2\pt1851_profile_may.png
Plot saved to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\StreetFacelifts\STREETFACELIFTS_P1\plots2\pt1851_profile_october.png
Plot saved to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\StreetFacelifts\STREETFACELIFTS_P1\plots2\pt2020_reductionheatmap.png
Plot saved to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\StreetFacelifts\STREETFACELIFTS_P1\plots2\pt2020_comfortheatmap.png
Plot saved to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\StreetFacelifts\STREETFACELIFTS_P1\plots2\pt2020_profile_may.png
Plot saved to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\StreetFacelifts\STREETFACELIFTS_P1\plots2\pt2020_profile_october.png
Summary saved to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\StreetFace